<a href="https://colab.research.google.com/github/ankita1200/Machine-Learning-Topics/blob/main/S%26P500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
import plotly.express as px
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.linear_model import LinearRegression

Get all the stock names

In [2]:
# Fetch S&P 500 tickers from Wikipedia
sp500_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_table = pd.read_html(sp500_url)
tickers = sp500_table[0]["Symbol"].tolist()

In [3]:
# Define S&P 500 stocks (Example: AAPL, MSFT, TSLA)
#tickers = ["AAPL", "MSFT", "TSLA", "GOOGL", "AMZN", "WFC", "YUM", "FICO"]

# Fetch historical stock prices for the last 2 years
data = yf.download(tickers, start="2023-01-01", end="2025-01-01", progress=False)['Close']

# Save data for analysis
data.to_csv("sp500_stock_data.csv")

# Display first few rows
print(data.tail())

YF.download() has changed argument auto_adjust default to True


ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')
ERROR:yfinance:['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-01-01 -> 2025-01-01)')


Ticker               A        AAPL        ABBV        ABNB         ABT  \
Date                                                                     
2024-12-24  135.848907  257.916443  178.318420  134.990005  114.160919   
2024-12-26  135.579407  258.735504  177.525894  135.320007  114.668251   
2024-12-27  135.289932  255.309296  176.347015  133.384995  114.389717   
2024-12-30  134.171997  251.923019  174.553925  131.809998  112.211151   
2024-12-31  134.339996  250.144974  176.039917  131.410004  112.519531   

Ticker           ACGL         ACN        ADBE         ADI        ADM  ...  \
Date                                                                  ...   
2024-12-24  92.669998  361.630005  447.940002  218.089996  50.069885  ...   
2024-12-26  92.930000  360.429993  450.160004  218.059998  50.050106  ...   
2024-12-27  92.339996  356.179993  446.480011  216.990005  50.020439  ...   
2024-12-30  91.889999  352.489990  445.799988  212.559998  49.516079  ...   
2024-12-31  92.3499

##Approach 1: Trend Slope (Linear Regression)
Fit a simple linear regression on stock prices over a rolling window.
If the slope is positive, classify as UP; if negative, classify as DOWN; if near zero, classify as FLAT.



In [4]:
def classify_stock_trend(prices, window=252):  # 252 trading days ≈ 1 year
    trends = []
    for stock in prices.columns:
        y = prices[stock].dropna().values.reshape(-1, 1)
        X = np.arange(len(y)).reshape(-1, 1)

        if len(y) < window:
            trends.append("flat")  # Not enough data
            continue

        model = LinearRegression().fit(X, y)
        slope = model.coef_[0][0]

        if slope > 0.05:
            trends.append("up")
        elif slope < -0.05:
            trends.append("down")
        else:
            trends.append("flat")

    return dict(zip(prices.columns, trends))

# Get classification labels
labels = classify_stock_trend(data)
print(labels)

{'A': 'flat', 'AAPL': 'up', 'ABBV': 'up', 'ABNB': 'up', 'ABT': 'flat', 'ACGL': 'up', 'ACN': 'up', 'ADBE': 'up', 'ADI': 'up', 'ADM': 'down', 'ADP': 'up', 'ADSK': 'up', 'AEE': 'flat', 'AEP': 'flat', 'AES': 'flat', 'AFL': 'up', 'AIG': 'up', 'AIZ': 'up', 'AJG': 'up', 'AKAM': 'flat', 'ALB': 'down', 'ALGN': 'down', 'ALL': 'up', 'ALLE': 'up', 'AMAT': 'up', 'AMCR': 'flat', 'AMD': 'up', 'AME': 'up', 'AMGN': 'up', 'AMP': 'up', 'AMT': 'up', 'AMZN': 'up', 'ANET': 'up', 'ANSS': 'up', 'AON': 'up', 'AOS': 'flat', 'APA': 'flat', 'APD': 'flat', 'APH': 'up', 'APO': 'up', 'APTV': 'down', 'ARE': 'flat', 'ATO': 'up', 'AVB': 'up', 'AVGO': 'up', 'AVY': 'up', 'AWK': 'flat', 'AXON': 'up', 'AXP': 'up', 'AZO': 'up', 'BA': 'down', 'BAC': 'flat', 'BALL': 'flat', 'BAX': 'flat', 'BBY': 'flat', 'BDX': 'flat', 'BEN': 'flat', 'BF.B': 'flat', 'BG': 'flat', 'BIIB': 'down', 'BK': 'up', 'BKNG': 'up', 'BKR': 'flat', 'BLDR': 'up', 'BLK': 'up', 'BMY': 'flat', 'BR': 'up', 'BRK.B': 'flat', 'BRO': 'up', 'BSX': 'up', 'BWA': 'flat

## 50-SMA and 200-SMA
1. Standard window sizes used for technical analysis
2.

In [15]:
sp500_index = yf.download("^GSPC", start="2022-01-01", end="2024-01-01", progress=False)["Close"]
print(sp500_index.iloc[-1].values[0])
print(sp500_index.iloc[0].values[0])

4769.830078125
4796.56005859375


In [17]:
actual_returns = (data.iloc[-1] - data.iloc[0]) / data.iloc[0]  # Compute actual return percentage

correct_predictions = 0
total_stocks = len(labels)

for stock, trend in labels.items():
    if trend == "up" and actual_returns[stock] > 0:
        correct_predictions += 1
    elif trend == "down" and actual_returns[stock] < 0:
        correct_predictions += 1
    elif trend == "flat" and abs(actual_returns[stock]) < 0.02:  # Flat if movement <2%
        correct_predictions += 1

accuracy = (correct_predictions / total_stocks) * 100
print(f"Adjusted Classification Accuracy: {accuracy:.2f}%")


Adjusted Classification Accuracy: 56.66%


##Moving Average technique

In [22]:
data.head()

Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-03,147.931229,123.632530,149.164108,84.900002,104.779053,59.393120,261.824402,336.920013,156.755081,83.634407,...,238.615921,28.467720,84.021248,65.245392,98.713249,107.810150,122.183739,125.222504,260.200012,143.661118
2023-01-04,149.538330,124.907715,150.367493,88.720001,106.337624,59.687901,260.933105,341.410004,160.093765,80.593163,...,239.890884,28.809036,89.213074,65.804085,99.000572,107.644676,123.839035,126.383438,262.989990,145.725311
2023-01-05,149.972153,123.583107,150.183777,87.709999,105.945602,59.849552,254.771637,328.440002,154.091812,78.578087,...,239.842209,27.951130,89.644897,64.472542,101.215607,104.588303,122.895889,124.041908,260.059998,142.291504
2023-01-06,145.594559,128.130249,152.994720,88.519997,107.408585,60.762417,260.807190,332.750000,159.717438,80.182671,...,246.645081,29.159578,91.833527,66.381378,102.438972,108.306557,125.388458,123.668045,273.420013,144.433990
2023-01-09,145.397369,128.654144,148.502716,89.239998,107.236450,59.592808,265.205444,341.980011,161.242065,79.641594,...,248.231445,28.984308,92.579430,67.023888,100.529778,108.812714,124.878387,124.012398,279.290009,143.866531


In [28]:
def classify_stock_ma(prices):
    trends = {}
    for stock in prices.columns:
      df = prices[stock].dropna()
      if len(df) < 200:
        trends[stock] = "flat"
        continue

      sma_50 = df.rolling(window=50).mean()
      sma_200 = df.rolling(window=200).mean()

      if sma_50.iloc[-1] > sma_200.iloc[-1]:
        trends[stock] = "up"
      elif sma_50.iloc[-1] < sma_200.iloc[-1]:
        trends[stock] = "down"
      else:
        trends[stock] = "flat"

    return trends

labels_ma = classify_stock_ma(data)
print(labels_ma)


{'A': 'down', 'AAPL': 'up', 'ABBV': 'up', 'ABNB': 'down', 'ABT': 'up', 'ACGL': 'down', 'ACN': 'up', 'ADBE': 'down', 'ADI': 'up', 'ADM': 'down', 'ADP': 'up', 'ADSK': 'up', 'AEE': 'up', 'AEP': 'up', 'AES': 'down', 'AFL': 'up', 'AIG': 'up', 'AIZ': 'up', 'AJG': 'up', 'AKAM': 'down', 'ALB': 'down', 'ALGN': 'down', 'ALL': 'up', 'ALLE': 'up', 'AMAT': 'down', 'AMCR': 'up', 'AMD': 'down', 'AME': 'up', 'AMGN': 'down', 'AMP': 'up', 'AMT': 'down', 'AMZN': 'up', 'ANET': 'up', 'ANSS': 'up', 'AON': 'up', 'AOS': 'down', 'APA': 'down', 'APD': 'up', 'APH': 'up', 'APO': 'up', 'APTV': 'down', 'ARE': 'down', 'ATO': 'up', 'AVB': 'up', 'AVGO': 'up', 'AVY': 'down', 'AWK': 'up', 'AXON': 'up', 'AXP': 'up', 'AZO': 'up', 'BA': 'down', 'BAC': 'up', 'BALL': 'down', 'BAX': 'down', 'BBY': 'up', 'BDX': 'down', 'BEN': 'down', 'BF.B': 'flat', 'BG': 'down', 'BIIB': 'down', 'BK': 'up', 'BKNG': 'up', 'BKR': 'up', 'BLDR': 'down', 'BLK': 'up', 'BMY': 'up', 'BR': 'up', 'BRK.B': 'flat', 'BRO': 'up', 'BSX': 'up', 'BWA': 'up', '

In [30]:
actual_returns = (data.iloc[-1] - data.iloc[0]) / data.iloc[0]  # Compute actual return percentage

correct_predictions = 0
total_stocks = len(labels_ma)

for stock, trend in labels_ma.items():
    if trend == "up" and actual_returns[stock] > 0:
        correct_predictions += 1
    elif trend == "down" and actual_returns[stock] < 0:
        correct_predictions += 1
    elif trend == "flat" and abs(actual_returns[stock]) < 0.02:  # Flat if movement <2%
        correct_predictions += 1

accuracy = (correct_predictions / total_stocks) * 100
print(f"Adjusted Classification Accuracy: {accuracy:.2f}%")


Adjusted Classification Accuracy: 80.91%


In [18]:
fig = px.line(data, x=data.index, y=data["AAPL"], title='AAPL Value')

fig.update_xaxes(rangeslider_visible=False,
                 rangeselector=dict(
                     buttons=list([
                         dict(count=6, label="6m", step="month", stepmode="backward"),
                         dict(count=1, label="1y", step="year", stepmode="backward"),
                         dict(count=2, label="2y", step="year", stepmode="backward"),
                         dict(step="all")])),
                 row=1,col=1)

fig.show()


In [22]:
fig = px.line(data, x=data.index, y=data["WFC"], title='AAPL Value')

fig.update_xaxes(rangeslider_visible=False,
                 rangeselector=dict(
                     buttons=list([
                         dict(count=6, label="6m", step="month", stepmode="backward"),
                         dict(count=1, label="1y", step="year", stepmode="backward"),
                         dict(count=2, label="2y", step="year", stepmode="backward"),
                         dict(step="all")])),
                 row=1,col=1)

fig.show()


In [21]:
fig = px.line(data, x=data.index, y=data["YUM"], title='AAPL Value')

fig.update_xaxes(rangeslider_visible=False,
                 rangeselector=dict(
                     buttons=list([
                         dict(count=6, label="6m", step="month", stepmode="backward"),
                         dict(count=1, label="1y", step="year", stepmode="backward"),
                         dict(count=2, label="2y", step="year", stepmode="backward"),
                         dict(step="all")])),
                 row=1,col=1)

fig.show()


In [7]:
def classify_stock_ma(prices):
    trends = {}
    for stock in prices.columns:
        df = prices[stock].dropna()
        sma_50 = df.rolling(window=50).mean()
        sma_200 = df.rolling(window=200).mean()

        if sma_50.iloc[-1] > sma_200.iloc[-1]:
            trends[stock] = "up"
        elif sma_50.iloc[-1] < sma_200.iloc[-1]:
            trends[stock] = "down"
        else:
            trends[stock] = "flat"

    return trends

labels_ma = classify_stock_ma(data)
print(labels_ma)


{'AAPL': 'up', 'AMZN': 'up', 'GOOGL': 'up', 'MSFT': 'up', 'TSLA': 'up'}


In [9]:
sp500_index = yf.download("^GSPC", start="2023-01-01", end="2025-01-01", progress=False)["Close"]
sp500_index

Ticker,^GSPC
Date,
2023-01-03,3824.139893
2023-01-04,3852.969971
2023-01-05,3808.100098
2023-01-06,3895.080078
2023-01-09,3892.090088
...,...
2024-12-24,6040.040039
2024-12-26,6037.589844
2024-12-27,5970.839844


In [ ]:
# Define overall market trend
market_trend = "up" if sp500_index[-1] > sp500_index[0] else "down"

# Compare classification with market
correct = sum(1 for stock, trend in labels.items() if trend == market_trend)
accuracy = correct / len(labels) * 100

print(f"Classification Accuracy: {accuracy:.2f}%")

In [10]:

# Define overall market trend
market_trend = sp500_index

# Compare classification with market
correct = sum(1 for stock, trend in labels.items() if trend == market_trend)
accuracy = correct / len(labels) * 100

print(f"Classification Accuracy: {accuracy:.2f}%")


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [12]:
sp500_index = yf.download("^GSPC", start="2022-01-01", end="2024-01-01", progress=False)["Close"]
sp500_index

Ticker,^GSPC
Date,
2022-01-03,4796.560059
2022-01-04,4793.540039
2022-01-05,4700.580078
2022-01-06,4696.049805
2022-01-07,4677.029785
...,...
2023-12-22,4754.629883
2023-12-26,4774.750000
2023-12-27,4781.580078


In [13]:
type(sp500_index)

pandas.core.frame.DataFrame

In [17]:
sp500_index.iloc[0,:].item()

4796.56005859375

In [4]:
from datetime import datetime
import yfinance as yf
import mplfinance as mpf
start_date = datetime(2019, 1, 1)
end_date = datetime(2019, 12, 31)
data = yf.download('AAPL', start=start_date, end=end_date)
mpf.plot(data,type='candle',mav=(3,6,9),volume=True,show_nontrading=True)


ModuleNotFoundError: No module named 'mplfinance'